Part 1 - Collect Data into CSVs to work with

In [42]:
import sys
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import time

In [3]:
load_dotenv()
load_dotenv('.env')
sports_data_api_key = os.getenv('SPORTS_DATA_API_KEY')

In [4]:
uri = 'https://api-nba-v1.p.rapidapi.com'
api_key_str = '?key='+sports_data_api_key

In [36]:
headers = {
	"X-RapidAPI-Key": sports_data_api_key,
	"X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com"
}

#seasons = requests.get(uri+'/seasons', headers=headers).json()

seasons = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

print(seasons)

[2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]


In [46]:
headers = {
	"X-RapidAPI-Key": sports_data_api_key,
	"X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com"
}

teams = requests.get(uri+'/teams', headers=headers).json()['response']

print(teams)

[{'id': 1, 'name': 'Atlanta Hawks', 'nickname': 'Hawks', 'code': 'ATL', 'city': 'Atlanta', 'logo': 'https://upload.wikimedia.org/wikipedia/fr/e/ee/Hawks_2016.png', 'allStar': False, 'nbaFranchise': True, 'leagues': {'standard': {'conference': 'East', 'division': 'Southeast'}, 'vegas': {'conference': 'summer', 'division': None}, 'utah': {'conference': 'East', 'division': 'Southeast'}, 'sacramento': {'conference': 'East', 'division': 'Southeast'}}}, {'id': 2, 'name': 'Boston Celtics', 'nickname': 'Celtics', 'code': 'BOS', 'city': 'Boston', 'logo': 'https://upload.wikimedia.org/wikipedia/fr/thumb/6/65/Celtics_de_Boston_logo.svg/1024px-Celtics_de_Boston_logo.svg.png', 'allStar': False, 'nbaFranchise': True, 'leagues': {'standard': {'conference': 'East', 'division': 'Atlantic'}, 'vegas': {'conference': 'summer', 'division': None}, 'utah': {'conference': 'East', 'division': 'Atlantic'}, 'sacramento': {'conference': 'East', 'division': 'Atlantic'}}}, {'id': 3, 'name': 'Brisbane Bullets', 'nic

In [53]:
team_names = ['New Orleans Pelicans', 'LA Clippers', 'Phoenix Suns', 'Los Angeles Lakers', 'Miami Heat', 'Orlando Magic', 'Milwaukee Bucks', 'Philadelphia 76ers','Oklahoma City Thunder', 'Dallas Mavericks', 'Minnesota Timberwolves', 'Denver Nuggets', 'Boston Celtics', 'Cleveland Cavaliers', 'Indiana Pacers', 'New York Knicks']
final_teams = []

for team in teams:
    
    if team['name'] in team_names:
        #print('Added:',team['name'])
        final_teams.append(team)

print(len(final_teams), len(team_names))

#confirm all names were found
for n in team_names:
    found = False
    for team in teams:
        if n == team['name']:
            found = True
            break;
    if found == False:
        print(n)



8 8


In [54]:
pd.json_normalize(final_teams, max_level=0).to_csv('Resources/All Teams.csv', index=False)

In [55]:
# test the new CSV as a DF

df_teams = pd.read_csv('Resources/All Teams.csv')
df_teams.head()

,id,name,nickname,code,city,logo,allStar,nbaFranchise,leagues
0,16,LA Clippers,Clippers,LAC,LA,https://upload.wikimedia.org/wikipedia/fr/d/d6...,False,True,"{'standard': {'conference': 'West', 'division'..."
1,17,Los Angeles Lakers,Lakers,LAL,Los Angeles,https://upload.wikimedia.org/wikipedia/commons...,False,True,"{'standard': {'conference': 'West', 'division'..."
2,20,Miami Heat,Heat,MIA,Miami,https://upload.wikimedia.org/wikipedia/fr/thum...,False,True,"{'standard': {'conference': 'East', 'division'..."
3,21,Milwaukee Bucks,Bucks,MIL,Milwaukee,https://upload.wikimedia.org/wikipedia/fr/3/34...,False,True,"{'standard': {'conference': 'East', 'division'..."
4,23,New Orleans Pelicans,Pelicans,NOP,New Orleans,https://upload.wikimedia.org/wikipedia/fr/thum...,False,True,"{'standard': {'conference': 'West', 'division'..."


In [56]:
headers = {
	"X-RapidAPI-Key": sports_data_api_key,
	"X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com"
}

def get_players(team_id, season):
    time.sleep(30)
    return requests.get(uri+'/players', headers=headers, params={"team":team_id, "season":season}).json()['response']

all_players = []

for team_id in df_teams['id']:
    for season in seasons:
        all_players.extend(
            get_players(team_id, season)
        )

print(len(all_players))

2013


In [59]:
pd.json_normalize(all_players, max_level=0).to_csv('Resources/All Players.csv', index=False)

# test the new CSV as a DF

df_players = pd.read_csv('Resources/All Players.csv')
df_players.tail()

,id,firstname,lastname,birth,nba,height,weight,college,affiliation,leagues
2008,153,Kevin,Durant,"{'date': '1988-09-29', 'country': 'USA'}","{'start': 2007, 'pro': 13}","{'feets': '6', 'inches': '10', 'meters': '2.08'}","{'pounds': '240', 'kilograms': '108.9'}",Texas,Texas/USA,"{'standard': {'jersey': 7, 'active': True, 'po..."
2009,570,Thaddeus,Young,"{'date': '1988-06-21', 'country': 'USA'}","{'start': 2007, 'pro': 14}","{'feets': '6', 'inches': '8', 'meters': '2.03'}","{'pounds': '235', 'kilograms': '106.6'}",Georgia Tech,Georgia Tech/USA,"{'standard': {'jersey': 21, 'active': True, 'p..."
2010,196,Eric,Gordon,"{'date': '1988-12-25', 'country': 'USA'}","{'start': 2008, 'pro': 13}","{'feets': '6', 'inches': '3', 'meters': '1.9'}","{'pounds': '215', 'kilograms': '97.5'}",Indiana,Indiana/USA,"{'standard': {'jersey': 10, 'active': True, 'p..."
2011,45,Bradley,Beal,"{'date': '1993-06-28', 'country': 'USA'}","{'start': 2012, 'pro': 9}","{'feets': '6', 'inches': '4', 'meters': '1.93'}","{'pounds': '207', 'kilograms': '93.9'}",Florida,Florida/USA,"{'standard': {'jersey': 3, 'active': True, 'po..."
2012,398,Jusuf,Nurkic,"{'date': '1994-08-23', 'country': 'Bosnia and ...","{'start': 2014, 'pro': 7}","{'feets': '6', 'inches': '11', 'meters': '2.11'}","{'pounds': '290', 'kilograms': '131.5'}",Cedevita,Cedevita/Bosnia and Herzegovina,"{'standard': {'jersey': 27, 'active': True, 'p..."


In [13]:
headers = {
	"X-RapidAPI-Key": sports_data_api_key,
	"X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com"
}

games = requests.get(uri+'/games', headers=headers).json()

print(games)

{'get': 'games/', 'parameters': [], 'errors': {'required': 'At least one parameter is required.'}, 'results': 0, 'response': []}


In [14]:
headers = {
	"X-RapidAPI-Key": sports_data_api_key,
	"X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com"
}

standings = requests.get(uri+'/standings', headers=headers).json()

print(standings)

{'get': 'standings/', 'parameters': [], 'errors': {'required': 'At least one parameter is required.'}, 'results': 0, 'response': []}


In [16]:
headers = {
	"X-RapidAPI-Key": sports_data_api_key,
	"X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com"
}

statistics = requests.get(uri+'/games/statistics', headers=headers).json()

print(statistics)

{'get': 'games/statistics', 'parameters': [], 'errors': {'required': 'At least one parameter is required.'}, 'results': 0, 'response': []}


Part 2 - ???